# Q-learning from scratch
The value of a state or a state-action pair is the expected return cost if we start from that state or state-action pair. Using value function, we can define the Bellman equationa and assort to temporal difference learning which is a powerful method in RL. In the sequel, We define the main component in the $Q$-learning algorithm.

Note that in $Q$ learning, we do not directly parameterize the policy. We parametrize the $Q$-function and the policy is defined by minimizing the $Q$-function. So, what we learn is the $Q$-function. The policy is obtained from the $Q$ function. 

## 1.  The $Q$- function 
The quality function is equal to the expected return reward for taking an arbitrary action $a$ and then following the policy $\pi$. The following equation defines the Bellman equation for the $Q$-function

\begin{align}
\large
Q^{\pi}(s,a) =r(s,a)+ \gamma\: \mathbf{E}_{\tau \sim \pi}[Q^{\pi}(s^{\prime}, \pi(s^{\prime}))]
\end{align}
where
* $\large \pi$ is the policy. We'll define it in 1.3. 
* $\large \tau$ is a trajectory sampled from the environment starting from $s$ and following the policy $\pi$. 
* $r_k=r(s_k,a_k)$ is the immediate reward.
* $0 \leq \gamma \leq 1$ is the forgetting factor.

<center>__In a $Q$-learning algorithm, we learn the $Q$ function.__ <center>

### 1.2 The policy
The policy is the action maximizes the expected return from starting in $s$

\begin{align}
\pi = \arg \max_a Q (s,a).
\end{align}

The optimal policy is denoted by $\pi^*$.

### 1.3 Optimal $Q$ function
The optimal $Q$ function is denoted by $Q^*$ and gives the expected reward cost if you start in state $s$, take an arbitrary action $a$, and then forever after act according to the optimal policy in the environment

\begin{align}
\large
Q^{*}(s,a) =r(s,a)+ \gamma \: \mathbf{E}_{\tau \sim \pi^*}[Q^{*}(s^{\prime}, \pi^*(s^{\prime}))].
\end{align}

## 2. $Q$ function network
As the name implies, in a $Q$-learning algorithm, we build a (possibly deep) network and learn the $Q$-function.
If this `network` represent the __true__ $Q$-function, then it satisfies the Bellman equation in section 1. Before learning however, the network does not represent the true $Q$ function. We define the _temporal difference error $e$_

\begin{align}
 e = c(s,a)+ \gamma\: Q^{\pi}(s^{\prime}, \pi(s^{\prime}))-Q^{\pi}(s,a)
\end{align}
where we have replaced $E[Q^{\pi}(s^{\prime},\pi(s^{\prime}))]$ with a stochastic sample. We learn the parameters in the $Q$ network to minimize the mean square (mse) $\dfrac{1}{2} \: e^2$. In the sequel, we show how to define the network and the error when the action space is discrete or continuous. So, depending on the problem that you are solving, you need to consider only one of the following structures.

### 2.1 Discrete action space
When there is a finite number of $n_a$ actions, the $Q$ function `network` takes the state `s` as the input and generate $n_a$ actions as the output. So the output is a vector with however many entries as there are actions, and the actions are the indices for the vector. $Q(s,a)$ is obtained by indexing into the output vector `network(s)`. 

The policy $\pi$ is the index which the output of the network is maximized.

Now, we define an mse cost for the network:

We have configured our network, by selecting the structure and cost function to be minmized. The last step is to feed the network with `states`, `actions`, `next_states`, and `dones` and update the parameters of the network. This is done by 

### 2.2 Continuous action space
When the action space is continuous, the $Q$ function `network` takes `s` and `a` as the input and generate a single value $Q(s,a)$ as the output. The optimal action $a^*$ is then obtained by mathematical minimization of the function $Q(s,a)$. For example, for a linear quadratic system, one can consider a quadratic $Q$-network

\begin{align}
Q(s,a) =\begin{bmatrix}
s^T & a^T
\end{bmatrix}\begin{bmatrix}
q_{ss} & q_{sa}\\
q_{sa}^T & q_{aa} \end{bmatrix}\begin{bmatrix}
s\\a \end{bmatrix}
\end{align}

Then, the optimal polisy that maximizes $Q(s,a)$ is given by $\dfrac{\partial Q(s,a)}{\partial a}=0$

\begin{align}
a^* = -q_{aa}^{-1}q_{sa}^T s.
\end{align}

The $Q$ function as denoted above is usually learned by Least Square Temporal Difference learning (LSTD).

## 3. How to select action $a$? _exploration vs. exploitation_
You have probably heard about _exploration vs. exploitation_. This concept is best described by this example. Suppose that you want to go to a restaurant in the town. Exploration means that you select a random restaurant that you have not tried before. Exploitation means that you go to your favourite one. The good point with exploitation is that you like what you'll eat and the good point with exploration is that you might find something that you like more than your favorite. 

The same thing happens in RL. If the agent only sticks to exploitation, it can never improve its policy and it will stuck in a local minimum forever. On the other hand, if the agent only explores, it never uses what it has learned and only tries random things. It is important to balance the levels of exploration and exploitation. The simplest way of selecting $a$ to have both exploration and exploitation is described here for discrete and continuous action space.

### 3.1 Discrete action space
When there is a finite number of actions, the action $a$ is selected as follows. We set a level $0<\epsilon<1$ (for example $\epsilon = 0.1$) and we select a random number $r\sim [0,\:1]$. If $r<\epsilon$, we explore by selecting a random action otherwise, we follow the policy by maximizing the $Q$ function 

\begin{align}
a = \begin{cases}
\text{random action}\quad \text{if   } r \:<\: \epsilon\\
\arg \max_a Q (s,a) \quad \text{Otherwise}
\end{cases}
\end{align}

The following code implement the above expression:


### 3.2 Continuous action space
When the action space is continuous, the action $a$ is selected as the optimal policy plus some randomness. Let $r \sim \mathcal{N}(0,\sigma)$

\begin{align}
a = \arg \max_a Q (s,a) + r
\end{align}

For example, in the linear quadratic case, the above expression can be coded as the following where $n_a$ is the dimension of the input


## 4. Putting all together
Now, we put all steps together to run $Q$-learning algorithm. 

First, we build a (deep) network to represent $Q(s,a)$= `network(s)`. See section 2. Then, we iteratively improve the network. In each iteration of the algorithm, we do the following
* i. We rollout the environment using the current policy by following these steps:
    * i.a. We initialize empty histories for `states=[]`, `actions=[]`, `rewards=[]`, `next_states=[]`, `dones=[]`.
    * i.b. We observe the `state` $s$ and select the `action` $a$. See section 3.
    * i.c. We derive the environment using $a$ and observe the `reward` $r$, the next state $s^{\prime}$, and the boolean $done$ (which is `True` if the episode has ended and `False` otherwise).
    * i.d. We add $s,\:a,\:r,\:s^{\prime},\:done$ to the history batch `states`, `actions`, `rewards`, `next_states`, `dones`.
    * i.e. We continue from i.b. until the episode ends.
* ii. We supply `states`, `actions`, `rewards`, `next_states`, `dones` to the network and optimize the parameters of the network. See section 2.1. 

## 5. Related files
* $Q$ learning with continuous action space: The linear quadratic (study and code)
* $Q$ learning with continuous action space: The linear quadratic (only code)
* [$Q$ learning with discrete action space: The cartpole example (study and code)](q_cartpole_notebook.ipynb)
* $Q$ learning with continuous action space: The cartpole example (only code)